In [30]:
import numpy as np
import scipy.stats as stats

In [31]:
Y = np.array([83, 85, 84, 85, 85, 86, 86, 87, 86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90, 89, 90, 90, 91, 90, 92])
KSI=np.zeros((len(Y),5))
KSI[0:2,0]=1
KSI[2:8,1]=1
KSI[8:19,2]=1
KSI[19:23,3]=1
KSI[23:25,4]=1
KSI, Y

(array([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.]]),
 array([83, 85, 84, 85, 85, 86, 86, 87, 86, 87, 87, 87, 88, 88, 88, 88, 88,
        89, 90, 89, 90, 90, 91, 90, 92]))

In [32]:
F = KSI.T.dot(KSI)
F_inv = np.linalg.inv(F)
def get_coef(ksi, nu):
    PSI_ = np.array([ksi[i] for i in range(ksi.shape[0])])
    Y_ = nu
    F_ = PSI_.T @ PSI_
    F_inv_ = np.linalg.inv(F_)
    betta_ = F_inv_ @ PSI_.T @ Y_
    return betta_
F_inv

array([[0.5       , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.16666667, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.09090909, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.25      , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.5       ]])

* a) Определить влияние возраста на содержание иммуноглобулина в крови с помощью регрессионного анализа

In [33]:
betta = get_coef(KSI, Y)
print(f'{np.round(betta[0], 2)}x1 + {np.round(betta[1], 2)}x2 + {np.round(betta[2], 2)}x3 + {np.round(betta[3], 2)}x4 + {np.round(betta[4], 2)}x5')

84.0x1 + 85.5x2 + 87.82x3 + 90.0x4 + 91.0x5


* b) Провести попарное сравнение средних в рамках регрессионной модели, с учетом множественности проверяемых гипотез

In [34]:
def prediction(betta, ksi):
    return betta @ ksi

In [35]:
def E(Y, betta, ksi):
    return Y - ([prediction(betta, ksi[i]) for i in range(ksi.shape[0])])
e = E(Y, betta, KSI)
print(e)

[-1.          1.         -1.5        -0.5        -0.5         0.5
  0.5         1.5        -1.81818182 -0.81818182 -0.81818182 -0.81818182
  0.18181818  0.18181818  0.18181818  0.18181818  0.18181818  1.18181818
  2.18181818 -1.          0.          0.          1.         -1.
  1.        ]


In [36]:
def RSS(e):
    return e.T@e
rss = RSS(e)
print(f'RSS={rss}')

RSS=23.136363636363637


In [37]:
p_values = np.array([])
for i in range (betta.shape[0]):
    for j in range(i + 1, betta.shape[0]):
        delta = (betta[i] - betta[j])/np.sqrt(rss*(F_inv[i, i] + F_inv[j, j])) * np.sqrt(KSI.shape[0] - betta.shape[0])
        p_value = stats.t.sf(np.abs(delta), KSI.shape[0] - betta.shape[0])
        p_values = np.append(p_values, p_value)
        result = 'различия не значимы' if p_value > 0.05 else 'различия значимы'
        p_value = np.round(p_value, 4)
        print(f'{i, j}: {p_value=} => \t{result}')
        

(0, 1): p_value=np.float64(0.0516) => 	различия не значимы
(0, 2): p_value=np.float64(0.0001) => 	различия значимы
(0, 3): p_value=np.float64(0.0) => 	различия значимы
(0, 4): p_value=np.float64(0.0) => 	различия значимы
(1, 2): p_value=np.float64(0.0002) => 	различия значимы
(1, 3): p_value=np.float64(0.0) => 	различия значимы
(1, 4): p_value=np.float64(0.0) => 	различия значимы
(2, 3): p_value=np.float64(0.0012) => 	различия значимы
(2, 4): p_value=np.float64(0.0005) => 	различия значимы
(3, 4): p_value=np.float64(0.1479) => 	различия не значимы


In [38]:
p_values = np.sort(p_values)
print(p_values)
m = p_values.shape[0]
for i in range(p_values.shape[0]):
    result
    if (p_values[i] <= 0.05 / m):
        result = 'отвергаем H0'
        m -= 1 
    else: 
        result = 'все хорошо'
    print(f'{i}: {result}')

[1.20628514e-06 1.27673625e-06 1.39105575e-06 2.04259155e-06
 8.31007244e-05 1.97520050e-04 5.01281453e-04 1.19666935e-03
 5.15502045e-02 1.47895676e-01]
0: отвергаем H0
1: отвергаем H0
2: отвергаем H0
3: отвергаем H0
4: отвергаем H0
5: отвергаем H0
6: отвергаем H0
7: отвергаем H0
8: все хорошо
9: все хорошо
